# 层和块
nn.Sequential定义了一种特殊的Module

In [5]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.0719, -0.0643, -0.0151, -0.0314,  0.0980,  0.0106,  0.0096,  0.0139,
          0.0171,  0.1358],
        [ 0.0283,  0.0251, -0.0501, -0.0921, -0.0039,  0.0901,  0.0652,  0.0480,
         -0.0977,  0.2386]], grad_fn=<AddmmBackward0>)

## 自定义块

In [2]:
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)  # 输出层

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

In [3]:
net = MLP()
net(X)

tensor([[-1.2792e-01, -2.0659e-01,  3.0194e-02, -8.7853e-02, -9.6134e-02,
         -7.7634e-02, -5.7369e-05, -1.5026e-01,  1.1569e-01,  1.2136e-01],
        [-7.4590e-02, -9.2572e-02, -2.7745e-02, -1.3042e-01, -1.6657e-01,
         -1.9541e-03, -5.4200e-02, -1.5756e-01,  4.3353e-02,  8.4797e-02]],
       grad_fn=<AddmmBackward0>)

## 顺序块

In [6]:
class MySequential(nn.Module) :     # 定义MySequential是nn.Module的一个子类
    def __init__(self, *args) :     # *args存放着每一层信息
        super().__init__()
        for block in args : 
            self._modules[block] = block    # _module是python中特殊的类，能存放每一层信息
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X
    
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0447,  0.0277,  0.0119,  0.1760,  0.1231,  0.0300,  0.0860, -0.0752,
          0.1193,  0.0070],
        [ 0.0514,  0.1013, -0.0087,  0.1560,  0.2576,  0.0240,  0.2003, -0.0912,
          0.0458, -0.0356]], grad_fn=<AddmmBackward0>)

## 在前向传播函数中执行代码

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.1254, grad_fn=<SumBackward0>)

## 混合搭配各种组合块的方法

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X) 

tensor(0.4380, grad_fn=<SumBackward0>)

# 参数管理

In [9]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))        # net0, net1, net2
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.0334],
        [0.0789]], grad_fn=<AddmmBackward0>)

In [13]:
# 参数访问
print(net[2].state_dict())      # 从自动机角度来讲，权重就是状态

print(type(net[2].bias))        # bias是一个Parameter的类型，是一个可以优化的参数
print(net[2].bias)
print(net[2].bias.data)
net[2].weight.grad == None
# .data是访问它的weight， .grad是访问梯度，因为没做反向计算，所以grad == None

OrderedDict([('weight', tensor([[ 0.0133,  0.2914,  0.0502, -0.1816,  0.2387,  0.0442, -0.2925, -0.0849]])), ('bias', tensor([0.0541]))])
<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.0541], requires_grad=True)
tensor([0.0541])


True

In [15]:
# 一次性访问所有参数
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])     # Relu是没有参数的，所以不会有输出

# 可以通过名字输出参数
net.state_dict()['2.bias'].data

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


tensor([0.0541])

In [19]:
def block1() : 
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())

def block2() :
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())      # 跟直接Sequential的区别是可以传进来每一个block的名字
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.1480],
        [0.1480]], grad_fn=<AddmmBackward0>)

In [20]:
# 看看网络是如何组织的
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


# 内置初始化

In [23]:
def init_normal(m) :    # m 会是一个module, normal是正态分布
    if type(m) == nn.Linear :   # 如果是全连接层
        nn.init.normal_(m.weight, mean = 0, std = 0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal)  # 对net里面的每一个layer调用这个函数
net[0].weight.data, net[0].bias.data

(tensor([[-0.0177,  0.0049, -0.0203,  0.0063],
         [ 0.0065,  0.0042, -0.0117,  0.0128],
         [-0.0140,  0.0037,  0.0063, -0.0125],
         [ 0.0178, -0.0140, -0.0034,  0.0008],
         [ 0.0014, -0.0211,  0.0060, -0.0230],
         [-0.0122, -0.0005, -0.0023, -0.0057],
         [ 0.0019, -0.0002, -0.0059,  0.0010],
         [ 0.0024, -0.0011, -0.0117,  0.0036]]),
 tensor([0., 0., 0., 0., 0., 0., 0., 0.]))

In [27]:
def init_constant(m) :    # m 会是一个module, normal是正态分布
    if type(m) == nn.Linear :   # 如果是全连接层
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)  # 对net里面的每一个layer调用这个函数
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

# 对某些块应用不同的初始化方法

In [29]:
def xavier(m) : 
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m) : 
    if type(m) == nn.Linear :
        nn.init.constant_(m.weight, 42)
        
net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.1672,  0.5539,  0.2150, -0.1902])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


# 自定义初始化

In [30]:
def my_init(m) : 
    if type(m) == nn.Linear:
        print("Init",
              *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5   # 只保留绝对值大于5的数，其他置0

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 7.9314, -6.1025, -0.0000, -7.2223],
        [ 0.0000, -0.0000,  0.0000,  0.0000]], grad_fn=<SliceBackward0>)

In [31]:
# 更加简单直接的方法
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000, -5.1025,  1.0000, -6.2223])

# 参数绑定

In [33]:
# 比如 ： 有些层想要共享权重
shared = nn.Linear(8, 8)
# 第2层和第4层是共享参数的线性层
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(), shared, nn.ReLU(), nn.Linear(8, 1))
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


# 自定义层
之前是讲如何自定义一个神经网络，接下来是如何构造一个没有任何参数的自定义层

In [1]:
import torch
import torch.nn.functional as F
from torch import nn
class CenteredLayer(nn.Module) : 
    def __init__(self):
        super().__init__()
    def forward(self, X):
        return X - X.mean()

layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [2]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

Y = net(torch.rand(4, 8))
Y.shape, Y.mean()

(torch.Size([4, 128]), tensor(6.6357e-09, grad_fn=<MeanBackward0>))

In [3]:
# 带参数的图层，通过parameter将参数包起来
class MyLinear(nn.Module) : 
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
        # randn, 按正态分布生成随机数
        
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)
dense = MyLinear(5, 3)
dense.weight

Parameter containing:
tensor([[-0.1291, -0.3252, -0.5894],
        [ 0.1511,  0.1947,  0.0958],
        [ 0.5759, -1.4857,  0.1512],
        [ 1.7532,  0.2908,  2.3265],
        [ 0.3193,  1.3574, -1.2215]], requires_grad=True)

In [4]:
# 使用自定义层直接执行正向传播计算
dense(torch.rand(2, 5))

tensor([[2.4808, 0.0000, 0.0000],
        [2.6881, 0.0253, 0.0000]])

In [5]:
# 使用自定义层构建模型
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.0000],
        [1.8241]])